In [1]:
import os
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
from ultralytics import YOLO
from concurrent.futures import ThreadPoolExecutor

In [2]:
from threading import Lock

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO("yolov8n.pt", verbose = False).to(device)
model_lock = Lock() 

In [3]:
# Function to detect persons in a single image
def detect_persons(image, model):
    with model_lock:
        results = model(image, verbose = False)
        boxes = np.array([box.xyxy.cpu().numpy() for box in results[0].boxes if box.cls.cpu().numpy()[0] == 0]).reshape(-1, 4)
        return boxes

# Function to create a single bounding box that encapsulates all detected persons
def get_combined_bounding_box(boxes):
    if len(boxes) == 0:
        return None
    x_min = np.min(boxes[:, 0])
    y_min = np.min(boxes[:, 1])
    x_max = np.max(boxes[:, 2])
    y_max = np.max(boxes[:, 3])
    return (x_min, y_min, x_max, y_max)

# Function to crop persons from an image
def crop_persons(image, combined_box):
    if combined_box is None:
        return None  # Return None if no person detected
    x1, y1, x2, y2 = combined_box
    crop = image.crop((x1, y1, x2, y2))
    return crop

# Function to process a single image
def process_image(image_path, output_dir, model, progress_bar, target_size=(640, 640)):
    image = Image.open(image_path)
    resized_image = image.resize(target_size)
    image_np = np.array(resized_image)
    
    boxes = detect_persons(image_np, model)
    combined_box = get_combined_bounding_box(boxes)
    crop = crop_persons(image, combined_box)
    
    if crop is not None:
        output_path = os.path.join(output_dir, os.path.basename(image_path))
        crop.save(output_path)
    progress_bar.update(1)

# Main pipeline function
def person_cropping_pipeline(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    image_paths = [os.path.join(input_dir, fname) for fname in os.listdir(input_dir) if fname.endswith(('.png', '.jpg', '.jpeg'))]

    with tqdm(total=len(image_paths)) as progress_bar:
        with ThreadPoolExecutor() as executor:
            for image_path in image_paths:
                executor.submit(process_image, image_path, output_dir, model, progress_bar)

In [5]:
from utils import SRC_DIR

input_dir = SRC_DIR
output_dir = SRC_DIR.as_posix().replace("images", "cropped_images")
person_cropping_pipeline(input_dir, output_dir)

(PosixPath('images'), 'cropped_images')